In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import data_generation as gen

# Estimation de la profondeur des téléséismes par apprentissage automatique à partir de signaux synthétiques

<div style="text-align: justify;">
La caractérisation de la profondeur d'un évènement sismique est un sujet important parmis les études menées sur les tremblements de terre : cela peut-être un critère de discrimination entre plusieurs types d'évènements (Laporte, 2022), ou encore simplement apporter des informations sur la nature géologique d'une zone d'étude. La densification et le renforcement de la couverture azimutale des réseaux sismologiques durant les dernières décennies permettent à présent d’optimiser la récupération pour un évènement de l’énergie des phases dites « de profondeur » (phases pP et sP), parfois détectables seulement dans certains azimuts. Ces phases permettent de déduire la profondeur de la source d’un évènement à distance télésismique (de 3000 à 10 000 km).
</div>

</br>

<div style="text-align: justify;">
Des méthodes récentes (p.ex. Blackwell et al, 2024) se portent sur une sommation de nombreux signaux télésismiques afin de maximiser l’énergie de ces phases par rapport au bruit, puis sur une double migration en profondeur selon les hypothèses pP et sP, permettant de co-valider les résultat obtenus. L'objectif de ce projet est d'estimer la profondeur de téléséismes directement à partir des signaux, ou plus particulièrement à partir d'une matrice correspondant à l'enveloppe énergétique de ces signaux (v. Figure 1), sans passer par cette étape de migration en profondeur qui peut être un peu lourde en calculs, en développant un modèle d'apprentissage automatique. Par manque de données facilement exploitables, le modèle a été entraîné sur des données synthétiques dont la génération est décrite ci-après.
</div>

<figure>
    <img src="notebook_images/real_energetic_envelope.png" style="width: 750px; display: block; margin: auto;" alt="Exemple des enveloppes énergétiques pour un évènement réel">
    <figcaption style="text-align: center;">Figure 1 : Enveloppes énergétiques pour un évènement sismique de magnitude 5 le 24 mai 2024 en Russie.</figcaption>
</figure>



## Génération des synthétiques

<div style="text-align: justify;">
Il s'agit de générer, pour un évènement aléatoire d'une profondeur maximale de 100 km, les signaux captés en 50 stations à distance télésismique, choisies aléatoirement également. Le modèle convolutif sera ensuite entraîné à partir de ces données pour plusieurs évènements. Les modules nécessaires pour la génération des signaux sont math, andom, obspy, numpy, matplotlib, scipy, ainsi que pytorch pour la génération des matrices sous forme de tenseurs compatibles pour l'entraînement du modèle.
</div>

#### Générer un signal unique

<div style="text-align: justify;">
On utilise le modèle de Terre 1D simplifié ak135 (Kennett, 2005) disponible dans TauP pour calculer les temps de trajets -- et donc les arrivées -- des ondes P, pP et sP à partir de coordonnées aléatoires pour la source (latitude, longitude, profondeur) et les stations (latitude et longitude, obligatoirement à distance télésismique par rapport à la source).
</div>

</br>

<div style="text-align: justify;">
Ensuite, on génère un signal de 60 s pour chacune de ces stations à partir des délais entre les arrivées des différentes phases, en plaçant un dirac pour chaque arrivée. Pour augmenter le réalisme du signal, la décroissance énergétique exponentielle des phases est simulée par l'ajout de diracs à intervalle régulier, d'amplitude de moins en moins importante. L'amplitude des phases est aléatoire (entre 0.5 et 1 pour la phase P, et de 0 à 110% de la phase P pour chacune des phases de profondeur), ainsi que leur signe original (qui peut également basculer), dans le but de simuler la radiation à la source. La génération des signaux est largement améliorable pour rendre les synthétiques plus réalistes, en particulier sur la manière de simuler la radiation à la source et la propagation des signaux.
</div>

</br>

<div style="text-align: justify;">
Ces signaux discrets sont ensuite convolués avec une ondelette de Ricker afin de générer un signal continu, auquel on ajoute du bruit blanc - avec un ratio signal sur bruit aléatoire compris entre 2 et 5 - avant de le filtrer entre 0.8 et 2.5 Hz (Laporte, 2022). Ce signal est enfin normalisé par Z-score avant d'en extraire l'enveloppe de Hilbert absolue, qui est elle-même normalisée par le maximum. On peut ici normaliser les signaux puisqu'on ne s'intéresse qu'à leur forme, afin de mettre en évidence les phases de profondeur. Les signaux sont ensuite décimés à 20 Hz, largement au-dessus de la fréquence de Nyquist ($ f_{\text{N}} = $ 5 Hz).
</div>

In [ ]:
signal, source, stations = gen.signal.generate_one_signal(plot=True)

#### Générer une matrice unique

<div style="text-align: justify;">
L'étape de génération du signal est répétée 50 fois afin de générer une matrice de dimensions (50, 1200) pour un même évènement. La profondeur est gardée en mémoire pour le calcul du misfit lors de l'entraînement, ainsi que les distances des stations à l'épicentre - pour éventuellement fournir plus d'informations au modèle.
</div>

In [ ]:
signal_matrix, depth, distances = gen.matrix.generate_matrix(num_stations=50, plot=True)

## Entraînement du modèle d'apprentissage automatique

#### Etape d'entraînement

In [ ]:
model_name, train_losses, val_losses, test_loss = gen.depth_model.train_DepthModel(model_name="Final_Model",
                                                                                   batch_size=256,
                                                                                   num_stations=50,
                                                                                   epochs=200,
                                                                                   include_distance=True,
                                                                                   plot=True
                                                                                  )

#### Résultats du modèle

In [ ]:
real_depth, predicted_depth, delta_depth = gen.depth_model.test_DepthModel(model_name="Final_Model",
                                                                           num_test=1000,
                                                                           num_stations=50,
                                                                           include_distance=True
                                                                          )

#### Utiliser un modèle existant

In [ ]:
depth = [50*1e3]  # Depths in meters
delta_depth = gen.depth_model.run_random_DepthModel(model_name="Final_Model",
                                             device_name="cuda",
                                             num_stations=50,
                                             include_distance=True,
                                             depth_list=depth,
                                             plot=True
                                            )